In [13]:
import numpy as np

class rbf:
    @staticmethod
    def radius(x0, x1):
        """
        Calculate the Euclidean distance (2-norm) between two points.
        
        Parameters:
        x0: A numpy.ndarray of size dim x 1 of the first point.
        x1: A numpy.ndarray of size dim x 1 of the second point.
        
        Returns:
        r: A float giving the distance between the two points.
        """
        return np.linalg.norm(x0 - x1, 2)
    
    @staticmethod
    def gaussian(x0, x1, eps):
        """
        Calculate the Gaussian kernel between two points.
        
        Parameters:
        x0: A numpy.ndarray of size dim x 1 of the first point.
        x1: A numpy.ndarray of size dim x 1 of the second point.
        eps: A float containing the shape parameter.
        
        Returns:
        phi: A float of the Gaussian kernel evaluated between two points.
        """
        r = rbf.radius(x0, x1)
        return np.exp(-(eps * r) ** 2)
    
    @staticmethod
    def mq(x0, x1, eps):
        """
        Calculate the Multiquadric kernel between two points.
        
        Parameters:
        x0: A numpy.ndarray of size dim x 1 of the first point.
        x1: A numpy.ndarray of size dim x 1 of the second point.
        eps: A float containing the shape parameter.
        
        Returns:
        phi: A float of the Multiquadric kernel evaluated between two points.
        """
        r = rbf.radius(x0, x1)
        return np.sqrt(1 + (eps * r) ** 2)
    
    @staticmethod
    def build_matrix(data_pts, eval_pts, kernel, fac, is_interpolation):
        """
        Build the appropriate matrix for RBF interpolation or weight calculation.
        
        Parameters:
        data_pts: A numpy.ndarray of size n x dim containing the locations of the known data.
        eval_pts: A numpy.ndarray of size q x dim containing the locations where to evaluate
                  the kernel with respect to the data points.
        kernel: The kernel function to use.
        fac: A float. The factor/parameter to pass into the kernel.
        is_interpolation: A boolean. True if the matrix to be returned is used for interpolation.
                          False if the matrix to be returned will be used to determine the RBF weights.
        
        Returns:
        M: A numpy.ndarray. This will be of size q x (n + dim + 1) if is_interpolation = True
           and of size (n + dim + 1) x (n + dim + 1) if is_interpolation = False.
        """
        n = data_pts.shape[0]
        dim = data_pts.shape[1]
        q = eval_pts.shape[0]
        
        # Build A matrix with kernel evaluations
        A = np.zeros((q, n))
        for i in range(q):
            for j in range(n):
                A[i, j] = kernel(eval_pts[i].reshape(-1, 1), data_pts[j].reshape(-1, 1), fac)
        
        # Build P matrix with polynomial terms
        P = np.ones((q, dim + 1))
        for i in range(q):
            P[i, 1:] = eval_pts[i]
        
        if is_interpolation:
            # For interpolation, return [A P]
            M = np.hstack((A, P))
        else:
            # For calculating weights, need to build a block matrix
            # [A  P]
            # [P' 0]
            P_transpose = np.ones((n, dim + 1))
            for i in range(n):
                P_transpose[i, 1:] = data_pts[i]
            
            zeros = np.zeros((dim + 1, dim + 1))
            
            # Use numpy.block to create the block matrix
            M = np.block([
                [A[:n], P_transpose],
                [P_transpose.T, zeros]
            ])
        
        return M
    
    @staticmethod
    def calculate_weights_mine(data_pts, data_vals, build_matrix_function, kernel, fac):
        """
        Calculate the interpolation weights.
    
        Parameters:
        data_pts: A numpy.ndarray of size n x dim containing the locations of the known data.
        data_vals: A numpy.ndarray of size n x 1 containing the function values at data_pts.
        build_matrix_function: A function. The function which is used to build the matrix.
        kernel: A function. The kernel function to use.
        fac: A float. The factor/parameter to pass into the kernel.
    
        Returns:
        wts: A numpy.ndarray. This will be of size (n + dim + 1) x 1 containing the interpolation weights.
        """
        dim = data_pts.shape[1]
    
        data_vals = data_vals.reshape(-1, 1)
    
        # Build system matrix for calculating weights
        mat = build_matrix_function(data_pts, data_pts, kernel, fac, False)
    
        # Prepare right-hand side vector
        lambda_mat = data_vals.copy()
        zeros = np.zeros((dim + 1, 1))
        lambda_mat = np.append(lambda_mat, zeros, axis=0)
    
        wts = np.linalg.solve(mat, lambda_mat)
    
        return wts
    
    @staticmethod
    def calculate_weights_proper(data_pts, data_vals, build_matrix_function, kernel, fac):
        """
        Calculate the interpolation weights.
        
        Parameters:
        data_pts: A numpy.ndarray of size n x dim containing the locations of the known data.
        data_vals: A numpy.ndarray of size n x 1 containing the function values at data_pts.
        build_matrix_function: A function. The function which is used to build the matrix.
        kernel: A function. The kernel function to use.
        fac: A float. The factor/parameter to pass into the kernel.
        
        Returns:
        wts: A numpy.ndarray. This will be of size (n + dim + 1) x 1 containing the interpolation weights.
        """
        n = data_pts.shape[0]
        dim = data_pts.shape[1]
        
        # Ensure data_vals is a column vector with the right shape
        data_vals = np.asarray(data_vals).reshape(n, 1)
        
        # Build the system matrix using the provided function
        A = build_matrix_function(data_pts, data_pts, kernel, fac, False)
        
        # Create the right-hand side vector [f, 0]^T
        b = np.zeros((n + dim + 1, 1))
        b[:n] = data_vals
        
        # Solve the linear system for the weights
        wts = np.linalg.solve(A, b)
        
        return wts
    
    @staticmethod
    def calculate_weights(data_pts, data_vals, build_matrix_function, kernel, fac):
        """
        Calculate the interpolation weights.
        Main implementation to be used for assignment.
        
        Parameters:
        data_pts: A numpy.ndarray of size n x dim containing the locations of the known data.
        data_vals: A numpy.ndarray of size n x 1 containing the function values at data_pts.
        build_matrix_function: A function. The function which is used to build the matrix.
        kernel: A function. The kernel function to use.
        fac: A float. The factor/parameter to pass into the kernel.
        
        Returns:
        wts: A numpy.ndarray. This will be of size (n + dim + 1) x 1 containing the interpolation weights.
        """
        # Using the proper implementation
        return rbf.calculate_weights_proper(data_pts, data_vals, build_matrix_function, kernel, fac)
    
    @staticmethod
    def interpolate(data_pts, eval_pts, kernel, fac, wts):
        """
        Evaluate the interpolant.
        
        Parameters:
        data_pts: A numpy.ndarray of size n x dim containing the locations of the known data.
        eval_pts: A numpy.ndarray of size q x dim containing the locations where to perform interpolation.
        kernel: A function. The kernel function to use.
        fac: A float. The factor/parameter to pass into the kernel.
        wts: A numpy.ndarray of size (n + dim + 1) x 1 of the pre-computed interpolation weights.
        
        Returns:
        f: A numpy.ndarray of q x 1 containing the interpolated values.
        """
        n = data_pts.shape[0]
        q = eval_pts.shape[0]
        
        # Build interpolation matrix
        M = rbf.build_matrix(data_pts, eval_pts, kernel, fac, True)
        
        # Compute interpolated values
        f = np.dot(M, wts)
        
        return f

In [16]:
def test_calculate_weights_comparison():
    """
    Extended test for both implementations of calculate_weights with 
    comprehensive test cases to thoroughly evaluate performance.
    """
    print("\n==== Testing and Comparing Both calculate_weights Implementations ====")
    
    test_cases = [
        # Original test cases
        {
            "name": "1D Parabola (3 points)",
            "data_pts": np.array([[-1.0], [0.0], [1.0]]),
            "data_vals": np.array([2.0, 0.0, 2.0]),
            "kernel": rbf.gaussian,
            "fac": 1.0
        },
        {
            "name": "2D Square (4 points)",
            "data_pts": np.array([[0.0, 0.0], [1.0, 0.0], [0.0, 1.0], [1.0, 1.0]]),
            "data_vals": np.array([0.0, 1.0, 1.0, 2.0]),
            "kernel": rbf.mq,
            "fac": 0.5
        },
        {
            "name": "3D Function (5 points)",
            "data_pts": np.array([
                [0.0, 0.0, 0.0],
                [1.0, 0.0, 0.0],
                [0.0, 1.0, 0.0],
                [0.0, 0.0, 1.0],
                [1.0, 1.0, 1.0]
            ]),
            "data_vals": np.array([0.0, 1.0, 1.0, 1.0, 3.0]),
            "kernel": rbf.gaussian,
            "fac": 1.0
        },
        
        # Additional test cases
        {
            "name": "1D Sine Function (10 points)",
            "data_pts": np.linspace(-np.pi, np.pi, 10).reshape(-1, 1),
            "data_vals": np.sin(np.linspace(-np.pi, np.pi, 10)),
            "kernel": rbf.gaussian,
            "fac": 0.5
        },
        {
            "name": "1D Polynomial (5 points)",
            "data_pts": np.array([[-2.0], [-1.0], [0.0], [1.0], [2.0]]),
            "data_vals": np.array([4.0, 1.0, 0.0, 1.0, 4.0]),  # x^2
            "kernel": rbf.mq,
            "fac": 0.3
        },
        {
            "name": "1D Exponential (7 points)",
            "data_pts": np.array([[-3.0], [-2.0], [-1.0], [0.0], [1.0], [2.0], [3.0]]),
            "data_vals": np.exp(np.array([-3.0, -2.0, -1.0, 0.0, 1.0, 2.0, 3.0])),
            "kernel": rbf.gaussian,
            "fac": 0.8
        },
        {
            "name": "2D Saddle Function (9 points)",
            "data_pts": np.array([
                [-1, -1], [-1, 0], [-1, 1],
                [0, -1], [0, 0], [0, 1],
                [1, -1], [1, 0], [1, 1]
            ]),
            "data_vals": np.array([
                1, 0, 1,
                0, -1, 0,
                1, 0, 1
            ]),  # x^2 - y^2
            "kernel": rbf.mq,
            "fac": 1.0
        },
        {
            "name": "2D Circle Function (12 points)",
            "data_pts": np.array([
                [1, 0], [0.866, 0.5], [0.5, 0.866], [0, 1],
                [-0.5, 0.866], [-0.866, 0.5], [-1, 0], [-0.866, -0.5],
                [-0.5, -0.866], [0, -1], [0.5, -0.866], [0.866, -0.5]
            ]),
            "data_vals": np.ones(12),  # Unit circle (all points at radius 1)
            "kernel": rbf.gaussian,
            "fac": 1.5
        },
        {
            "name": "2D Gaussian Peak (25 points)",
            "data_pts": np.array([[x, y] for x in np.linspace(-2, 2, 5) for y in np.linspace(-2, 2, 5)]),
            "data_vals": np.array([np.exp(-(x**2 + y**2)) for x, y in 
                           np.array([[x, y] for x in np.linspace(-2, 2, 5) for y in np.linspace(-2, 2, 5)])]),
            "kernel": rbf.mq,
            "fac": 0.7
        },
        {
            "name": "3D Sphere Function (26 points)",
            "data_pts": np.array([
                # Vertices of a cube
                [1, 1, 1], [1, 1, -1], [1, -1, 1], [1, -1, -1],
                [-1, 1, 1], [-1, 1, -1], [-1, -1, 1], [-1, -1, -1],
                # Points on coordinate axes
                [2, 0, 0], [-2, 0, 0], [0, 2, 0], [0, -2, 0], [0, 0, 2], [0, 0, -2],
                # Additional points
                [1, 1, 0], [1, 0, 1], [0, 1, 1], [-1, 1, 0], [-1, 0, 1], [0, -1, 1],
                [1, -1, 0], [1, 0, -1], [0, 1, -1], [-1, -1, 0], [-1, 0, -1], [0, -1, -1]
            ]),
            "data_vals": np.array([
                np.sqrt(x**2 + y**2 + z**2) for x, y, z in [
                    [1, 1, 1], [1, 1, -1], [1, -1, 1], [1, -1, -1],
                    [-1, 1, 1], [-1, 1, -1], [-1, -1, 1], [-1, -1, -1],
                    [2, 0, 0], [-2, 0, 0], [0, 2, 0], [0, -2, 0], [0, 0, 2], [0, 0, -2],
                    [1, 1, 0], [1, 0, 1], [0, 1, 1], [-1, 1, 0], [-1, 0, 1], [0, -1, 1],
                    [1, -1, 0], [1, 0, -1], [0, 1, -1], [-1, -1, 0], [-1, 0, -1], [0, -1, -1]
                ]
            ]),  # Radius from origin
            "kernel": rbf.gaussian,
            "fac": 0.5
        },
        {
            "name": "4D Hypercube (16 points)",
            "data_pts": np.array([
                [0, 0, 0, 0], [0, 0, 0, 1], [0, 0, 1, 0], [0, 0, 1, 1],
                [0, 1, 0, 0], [0, 1, 0, 1], [0, 1, 1, 0], [0, 1, 1, 1],
                [1, 0, 0, 0], [1, 0, 0, 1], [1, 0, 1, 0], [1, 0, 1, 1],
                [1, 1, 0, 0], [1, 1, 0, 1], [1, 1, 1, 0], [1, 1, 1, 1]
            ]),
            "data_vals": np.array(range(16)),  # Sequential values
            "kernel": rbf.gaussian,
            "fac": 2.0
        },
        {
            "name": "Edge Case: Closely Spaced Points (1D)",
            "data_pts": np.array([[0.0], [0.001], [0.002], [1.0], [1.001]]),
            "data_vals": np.array([0.0, 0.001, 0.002, 1.0, 1.001]),  # Linear function
            "kernel": rbf.gaussian,
            "fac": 10.0  # Higher shape parameter needed for close points
        },
        {
            "name": "Edge Case: Widely Spaced Points (1D)",
            "data_pts": np.array([[-100.0], [-10.0], [0.0], [10.0], [100.0]]),
            "data_vals": np.array([10000.0, 100.0, 0.0, 100.0, 10000.0]),  # x^2 function
            "kernel": rbf.mq,
            "fac": 0.01  # Lower shape parameter needed for widely spaced points
        }
    ]
    
    print(f"Running {len(test_cases)} test cases...")
    
    # Initialize result counters
    mine_passed = 0
    proper_passed = 0
    both_match = 0
    
    # Tables for results summary
    results_table = {
        "name": [],
        "points": [],
        "dim": [],
        "mine_error": [],
        "proper_error": [],
        "mine_time": [],
        "proper_time": [],
        "weight_diff": [],
        "both_pass": []
    }
    
    for i, test_case in enumerate(test_cases):
        print(f"\nTest Case {i+1}: {test_case['name']}")
        data_pts = test_case["data_pts"]
        data_vals = test_case["data_vals"]
        kernel = test_case["kernel"]
        fac = test_case["fac"]
        
        n = data_pts.shape[0]
        dim = data_pts.shape[1]
        
        print(f"Points: {n}, Dimensions: {dim}")
        
        # Initialize test case result variables
        mine_result = {"passed": False, "error": np.nan, "time": np.nan}
        proper_result = {"passed": False, "error": np.nan, "time": np.nan}
        weight_diff = np.nan
        
        # Test first implementation
        try:
            start_time = time.time()
            wts_mine = rbf.calculate_weights_mine(data_pts, data_vals, rbf.build_matrix, kernel, fac)
            mine_result["time"] = time.time() - start_time
            
            # Verify with interpolation
            reconstructed_mine = rbf.interpolate(data_pts, data_pts, kernel, fac, wts_mine)
            mine_result["error"] = np.max(np.abs(reconstructed_mine.flatten() - data_vals))
            mine_result["passed"] = mine_result["error"] < 1e-10
            
            if mine_result["passed"]:
                mine_passed += 1
            
            print(f"Mine - Shape: {wts_mine.shape}, Max Error: {mine_result['error']:.2e}, Time: {mine_result['time']:.6f}s")
            print(f"Mine - Test passed: {mine_result['passed']}")
        except Exception as e:
            print(f"Mine - Error: {str(e)}")
        
        # Test second implementation
        try:
            start_time = time.time()
            wts_proper = rbf.calculate_weights_proper(data_pts, data_vals, rbf.build_matrix, kernel, fac)
            proper_result["time"] = time.time() - start_time
            
            # Verify with interpolation
            reconstructed_proper = rbf.interpolate(data_pts, data_pts, kernel, fac, wts_proper)
            proper_result["error"] = np.max(np.abs(reconstructed_proper.flatten() - data_vals))
            proper_result["passed"] = proper_result["error"] < 1e-10
            
            if proper_result["passed"]:
                proper_passed += 1
            
            print(f"Proper - Shape: {wts_proper.shape}, Max Error: {proper_result['error']:.2e}, Time: {proper_result['time']:.6f}s")
            print(f"Proper - Test passed: {proper_result['passed']}")
        except Exception as e:
            print(f"Proper - Error: {str(e)}")
        
        # Compare results if both succeeded
        if 'wts_mine' in locals() and 'wts_proper' in locals():
            weight_diff = np.max(np.abs(wts_mine - wts_proper))
            implementations_match = weight_diff < 1e-10
            if implementations_match:
                both_match += 1
            print(f"Max difference between weights: {weight_diff:.2e}")
            print(f"Implementations match: {implementations_match}")
        
        # Add to results table
        results_table["name"].append(test_case["name"])
        results_table["points"].append(n)
        results_table["dim"].append(dim)
        results_table["mine_error"].append(mine_result["error"])
        results_table["proper_error"].append(proper_result["error"])
        results_table["mine_time"].append(mine_result["time"])
        results_table["proper_time"].append(proper_result["time"])
        results_table["weight_diff"].append(weight_diff)
        results_table["both_pass"].append(mine_result["passed"] and proper_result["passed"])
        
        # Clear variables to avoid confusion in next iteration
        if 'wts_mine' in locals():
            del wts_mine
        if 'wts_proper' in locals():
            del wts_proper
    
    # Print summary
    print("\n==== Summary of Test Results ====")
    print(f"Total test cases: {len(test_cases)}")
    print(f"Mine implementation passed: {mine_passed}/{len(test_cases)} ({mine_passed/len(test_cases)*100:.1f}%)")
    print(f"Proper implementation passed: {proper_passed}/{len(test_cases)} ({proper_passed/len(test_cases)*100:.1f}%)")
    print(f"Both implementations match: {both_match}/{len(test_cases)} ({both_match/len(test_cases)*100:.1f}%)")
    
    # Calculate average times and errors
    avg_mine_time = np.nanmean(results_table["mine_time"])
    avg_proper_time = np.nanmean(results_table["proper_time"])
    avg_mine_error = np.nanmean(results_table["mine_error"])
    avg_proper_error = np.nanmean(results_table["proper_error"])
    
    print(f"\nAverage execution time:")
    print(f"  Mine implementation: {avg_mine_time:.6f}s")
    print(f"  Proper implementation: {avg_proper_time:.6f}s")
    print(f"  Speed difference: {(avg_mine_time/avg_proper_time - 1)*100:.2f}%")
    
    print(f"\nAverage error:")
    print(f"  Mine implementation: {avg_mine_error:.2e}")
    print(f"  Proper implementation: {avg_proper_error:.2e}")
    
    # Identify problematic test cases
    if mine_passed < len(test_cases):
        print("\nTest cases where Mine implementation failed:")
        for i, passed in enumerate(results_table["mine_error"]):
            if np.isnan(passed) or passed >= 1e-10:
                print(f"  - {results_table['name'][i]} ({results_table['points'][i]} points, {results_table['dim'][i]}D)")
    
    if proper_passed < len(test_cases):
        print("\nTest cases where Proper implementation failed:")
        for i, passed in enumerate(results_table["proper_error"]):
            if np.isnan(passed) or passed >= 1e-10:
                print(f"  - {results_table['name'][i]} ({results_table['points'][i]} points, {results_table['dim'][i]}D)")
    
    if both_match < len(test_cases):
        print("\nTest cases where implementations don't match:")
        for i, diff in enumerate(results_table["weight_diff"]):
            if np.isnan(diff) or diff >= 1e-10:
                print(f"  - {results_table['name'][i]} ({results_table['points'][i]} points, {results_table['dim'][i]}D)")
    
    # Check for performance patterns
    print("\nPerformance analysis:")
    
    # Check performance by dimension
    dimensions = sorted(set(results_table["dim"]))
    for dim in dimensions:
        dim_indices = [i for i, d in enumerate(results_table["dim"]) if d == dim]
        if len(dim_indices) > 0:
            mine_times = [results_table["mine_time"][i] for i in dim_indices if not np.isnan(results_table["mine_time"][i])]
            proper_times = [results_table["proper_time"][i] for i in dim_indices if not np.isnan(results_table["proper_time"][i])]
            
            if len(mine_times) > 0 and len(proper_times) > 0:
                avg_mine = np.mean(mine_times)
                avg_proper = np.mean(proper_times)
                print(f"  {dim}D data: Mine avg {avg_mine:.6f}s, Proper avg {avg_proper:.6f}s, Ratio: {avg_mine/avg_proper:.2f}x")
    
    # Check performance by number of points
    point_ranges = [(0, 5), (6, 10), (11, 20), (21, 100)]
    for min_pts, max_pts in point_ranges:
        pt_indices = [i for i, p in enumerate(results_table["points"]) if min_pts <= p <= max_pts]
        if len(pt_indices) > 0:
            mine_times = [results_table["mine_time"][i] for i in pt_indices if not np.isnan(results_table["mine_time"][i])]
            proper_times = [results_table["proper_time"][i] for i in pt_indices if not np.isnan(results_table["proper_time"][i])]
            
            if len(mine_times) > 0 and len(proper_times) > 0:
                avg_mine = np.mean(mine_times)
                avg_proper = np.mean(proper_times)
                print(f"  {min_pts}-{max_pts} points: Mine avg {avg_mine:.6f}s, Proper avg {avg_proper:.6f}s, Ratio: {avg_mine/avg_proper:.2f}x")
    
    # Overall recommendation
    print("\nConclusion:")
    if proper_passed > mine_passed:
        print("The Proper implementation is more reliable and should be used.")
    elif mine_passed > proper_passed:
        print("The Mine implementation is more reliable and should be used.")
    else:
        if avg_proper_time < avg_mine_time:
            print("Both implementations have similar reliability, but the Proper implementation is faster.")
        elif avg_mine_time < avg_proper_time:
            print("Both implementations have similar reliability, but the Mine implementation is faster.")
        else:
            print("Both implementations have similar reliability and performance.")

In [17]:
test_calculate_weights_comparison()


==== Testing and Comparing Both calculate_weights Implementations ====
Running 13 test cases...

Test Case 1: 1D Parabola (3 points)
Points: 3, Dimensions: 1
Mine - Shape: (5, 1), Max Error: 6.66e-16, Time: 0.009245s
Mine - Test passed: True
Proper - Shape: (5, 1), Max Error: 6.66e-16, Time: 0.000346s
Proper - Test passed: True
Max difference between weights: 0.00e+00
Implementations match: True

Test Case 2: 2D Square (4 points)
Points: 4, Dimensions: 2
Mine - Shape: (7, 1), Max Error: 0.00e+00, Time: 0.000562s
Mine - Test passed: True
Proper - Shape: (7, 1), Max Error: 0.00e+00, Time: 0.000514s
Proper - Test passed: True
Max difference between weights: 0.00e+00
Implementations match: True

Test Case 3: 3D Function (5 points)
Points: 5, Dimensions: 3
Mine - Shape: (9, 1), Max Error: 0.00e+00, Time: 0.000693s
Mine - Test passed: True
Proper - Shape: (9, 1), Max Error: 0.00e+00, Time: 0.000661s
Proper - Test passed: True
Max difference between weights: 0.00e+00
Implementations match: T